## Model niet juist maar lukt niet met pooler output
zelf proberen een array te maken van alleen de last hidden state?

In [93]:
import glob
import logging
import os
import pickle
import random
import re
import shutil
import copy
from typing import Dict, List, Tuple

import nltk
import numpy as np
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange

from transformers import (MODEL_WITH_LM_HEAD_MAPPING, WEIGHTS_NAME, AdamW, AutoConfig, AutoModelWithLMHead, AutoTokenizer, PreTrainedModel, PreTrainedTokenizer, get_linear_schedule_with_warmup, BertModel)

try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

In [94]:
class Args():

    model_type='bert'
    #model_name_or_path="bert-base-uncased"
    model_name_or_path='./debiased_models/bert/debiased-checkpoint-best'
    cache_dir=None
    run_id=1
    device=-1
    batch_size=32

args=Args()

In [95]:
def open_file(file_path):
    with open(file_path) as f:
        output_file=f.read().splitlines()

    return output_file

def encode_all_lines(data):
    encoded_data=[]
    for i in range(len(data)):
        encoded_data.append(torch.tensor(tokenizer.encode(data[i], add_special_tokens=True)))
    return encoded_data

In [96]:
device="cpu" if args.device==-1 else "cuda:{}".format(args.device)

config=AutoConfig.from_pretrained(args.model_name_or_path, cache_dir=args.cache_dir)
config.output_hidden_states='true'

tokenizer=AutoTokenizer.from_pretrained(args.model_name_or_path, cache_dir=args.cache_dir)

block_size=tokenizer.model_max_length

#model=AutoModelWithLMHead.from_pretrained(args.model_name_or_path, from_tf=bool(".ckpt" in args.model_name_or_path), config=config, cache_dir=args.cache_dir)

model=BertModel.from_pretrained(args.model_name_or_path, from_tf=bool(".ckpt" in args.model_name_or_path), config=config, cache_dir=args.cache_dir)

model.to(device)
model.eval()

rand_seed=args.run_id

Some weights of the model checkpoint at ./debiased_models/bert/debiased-checkpoint-best were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at ./debiased_models/bert/debiased-checkpoint-best and are newly initialized: ['

In [97]:
if not os.path.exists("encodings"):
    os.makedirs("encodings")

In [98]:
gender_attr=open_file('./attribute_target_words/attributes/gender.txt')
occupation_stereo_target=open_file('./attribute_target_words/targets/gender_stereotype.txt')

In [99]:
def encode(nlp_model, nlp_tokenizer, texts, args):

    all_H=[]
    nlp_model.eval()
    with torch.no_grad():

        print("Encoding...")
        batch_size=args.batch_size
        pbar=tqdm(range(len(texts)), ascii=True)

        for i in range(0, len(texts)-batch_size, batch_size):

            batch_texts=texts[i: i+batch_size]

            batch_encoding=nlp_tokenizer.batch_encode_plus(batch_texts, padding=True, max_length=512, truncation=True)
            input_ids, token_type_ids, attention_mask=batch_encoding["input_ids"], batch_encoding["token_type_ids"], batch_encoding["attention_mask"]
            input_ids=torch.tensor(input_ids).to(device)
            token_type_ids=torch.tensor(token_type_ids).to(device)
            attention_mask=torch.tensor(attention_mask).to(device)
            H=nlp_model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask).pooler_output
            #H=np.array(nlp_model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask).hidden_states)
            assert len(H.shape)==2
            all_H.append(H.detach().cpu().numpy())

            pbar.update(batch_size)

        remaining=texts[(len(texts)//args.batch_size)*args.batch_size:]
        if len(remaining)>0:
            batch_encoding=nlp_tokenizer.batch_encode_plus(remaining, padding=True, max_length=512, truncation=True)
            input_ids, token_type_ids, attention_mask = batch_encoding["input_ids"], batch_encoding["token_type_ids"], batch_encoding["attention_mask"]
            input_ids = torch.tensor(input_ids).to(device)
            token_type_ids = torch.tensor(token_type_ids).to(device)
            attention_mask = torch.tensor(attention_mask).to(device)
            H = nlp_model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask).pooler_output
            assert len(H.shape) == 2
            all_H.append(H.detach().cpu().numpy())

    H_np=np.concatenate(all_H)
    assert len(H_np.shape)==2
    assert len(H_np)==len(texts)
    return H_np

In [100]:
H=encode(model, tokenizer, gender_attr, args)

Encoding...


100%|#########9| 448/450 [00:02<00:00, 151.93it/s]
